In [13]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import glob
import os
import shutil
from pyts.image import GramianAngularField
import cv2

In [ ]:
path = "/content/drive/MyDrive/Ju/"
csv_files = glob.glob(os.path.join(path, "*.txt"))

folder_path = '/content/drive/MyDrive/ju_detection_1/JU_CLASSIFICATION_3K_TO_10K_imbalanced'

# Iterate through the contents of the folder and delete them
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        os.remove(file_path)
    elif os.path.isdir(file_path):
        shutil.rmtree(file_path)


os.makedirs('/content/drive/MyDrive/ju_detection_1/JU_CLASSIFICATION_3K_TO_10K_imbalanced/PD')
os.makedirs('/content/drive/MyDrive/ju_detection_1/JU_CLASSIFICATION_3K_TO_10K_imbalanced/CO')
Pt_count = 0
Co_count = 0
error_count1 = 0
error_count2 = 0

In [24]:
# c_nt = 0
for f in csv_files:

    data = pd.read_csv(f, sep="\t",
                       names=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r',
                              's'])

    s_p = 3000  # start point
    e_p = 10000  # end point

    x = data.iloc[s_p:e_p, 0]
    y_17 = data.iloc[s_p:e_p, 17]  # for showing 17th column associated with first 8 column
    # y_17 = data.iloc[s_p:e_p, 18]  # for showing 18th column associated with last 8 columns
    y_17 = 2 * ((y_17 - y_17.min()) / (y_17.max() - y_17.min())) - 1
    y_17[y_17 < -0.25] = -1
    index_data, _ = find_peaks(-np.array(y_17), height=0.5)
    index_17 = x.iloc[index_data]
    index_17 = index_17.index.to_list()


    y_18 = data.iloc[s_p:e_p, 18]
    y_18 = 2 * ((y_18 - y_18.min()) / (y_18.max() - y_18.min())) - 1
    y_18[y_18 < -0.25] = -1
    index_data1, _ = find_peaks(-np.array(y_18), height= 0.5)
    index_18 = x.iloc[index_data1]
    index_18 = index_18.index.to_list()

    k = min(len(index_data),len(index_data1))

#     c_nt = c_nt+1
#     print(len(index_data), len(index_data1),c_nt)

    new_dir = f.rsplit('\\')[1]
    new_dir = new_dir[0:9]



    for i in range(k -1):
        gaf_image_left = []
        gaf_image_right = []
        for j in range(1, 9):
            y_l = data.iloc[index_17[i]:index_17[i + 1] - 1, j]
            y_l_normalized = 2 * ((y_l - y_l.min()) / (y_l.max() - y_l.min())) - 1
            y_l_normalized = y_l_normalized.values.reshape(1, -1)
            gaf = GramianAngularField(method='summation')
            try:
                gaf_image = gaf.fit_transform(y_l_normalized)
            except ValueError:
                error_count1 = error_count1+1
                pass
            
            gaf_image_array = cv2.resize(gaf_image[0], (100, 100))
            gaf_image_left.append(gaf_image_array)

            y_r = data.iloc[index_18[i]:index_18[i + 1] - 1, j]
            y_r_normalized = 2 * ((y_r - y_r.min()) / (y_r.max() - y_r.min())) - 1
            y_r_normalized = y_r_normalized.values.reshape(1, -1)
            gaf1 = GramianAngularField(method='summation')
            try:
                gaf_image1 = gaf1.fit_transform(y_r_normalized)
            except ValueError:
                error_count2 = error_count2 + 1
                pass
           
            gaf_image_array1 = cv2.resize(gaf_image1[0], (100, 100))
            gaf_image_right.append(gaf_image_array1)



        images = gaf_image_left + gaf_image_right
        rows = []
        for i in range(0, len(images), 4):
            row = cv2.hconcat(images[i:i + 4])
            rows.append(row)

        # Concatenate the rows vertically to create the final image
        final_image = cv2.vconcat(rows)
        final_image = final_image*255

        # Display the final image
        # cv2.imshow('Final Image', final_image)
        # cv2.waitKey(0)



        if "Pt" in new_dir:
            cv2.imwrite('/content/drive/MyDrive/ju_detection_1/JU_CLASSIFICATION_3K_TO_10K_imbalanced/PD/image_PD_'+str(Pt_count)+'.png', final_image)
            Pt_count += 1
        else:
            cv2.imwrite('/content/drive/MyDrive/ju_detection_1/JU_CLASSIFICATION_3K_TO_10K_imbalanced/CO/image_CO_'+str(Co_count)+'.png', final_image)
            Co_count += 1

print(error_count1)
print(error_count2)

53
10
